In [11]:
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras import backend
import os
import torch
import torch.utils.data
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from math import sin, cos
from PIL import ImageDraw, Image
import cv2
sys.path.append(os.path.abspath("./Residual-Attention-Network/"))
from model.residual_attention_network import ResidualAttentionModel_56 as ResidualAttentionModel
import cv2
import time

In [2]:
train = pd.read_csv('./train.csv')
# k is camera instrinsic matrix
k = np.array([[2304.5479, 0,  1686.2379],
           [0, 2305.8757, 1354.9849],
           [0, 0, 1]], dtype = np.float32)

In [3]:
# convert euler angle to rotation matrix
def euler_to_Rot(yaw, pitch, roll):
    Y = np.array([[cos(yaw), 0, sin(yaw)],
                  [0, 1, 0],
                  [-sin(yaw), 0, cos(yaw)]])
    P = np.array([[1, 0, 0],
                  [0, cos(pitch), -sin(pitch)],
                  [0, sin(pitch), cos(pitch)]])
    R = np.array([[cos(roll), -sin(roll), 0],
                  [sin(roll), cos(roll), 0],
                  [0, 0, 1]])
    return np.dot(Y, np.dot(P, R))
def rotate(x, angle):
    x = x + angle
    x = x - (x + np.pi) // (2 * np.pi) * 2 * np.pi
    return x

In [4]:
def draw_line(image, points):
    color = (255, 0, 0)
    cv2.line(image, tuple(points[1][:2]), tuple(points[2][:2]), color, 16)
    cv2.line(image, tuple(points[1][:2]), tuple(points[4][:2]), color, 16)

    cv2.line(image, tuple(points[1][:2]), tuple(points[5][:2]), color, 16)
    cv2.line(image, tuple(points[2][:2]), tuple(points[3][:2]), color, 16)
    cv2.line(image, tuple(points[2][:2]), tuple(points[6][:2]), color, 16)
    cv2.line(image, tuple(points[3][:2]), tuple(points[4][:2]), color, 16)
    cv2.line(image, tuple(points[3][:2]), tuple(points[7][:2]), color, 16)

    cv2.line(image, tuple(points[4][:2]), tuple(points[8][:2]), color, 16)
    cv2.line(image, tuple(points[5][:2]), tuple(points[8][:2]), color, 16)

    cv2.line(image, tuple(points[5][:2]), tuple(points[6][:2]), color, 16)
    cv2.line(image, tuple(points[6][:2]), tuple(points[7][:2]), color, 16)
    cv2.line(image, tuple(points[7][:2]), tuple(points[8][:2]), color, 16)
    return image


def draw_points(image, points):
    image = np.array(image)
    for (p_x, p_y) in points:
        # print("p_x, p_y", p_x, p_y)
        cv2.circle(image, (p_x, p_y), 5, (255, 0, 0), -1)
    return image

In [5]:
# image coordinate to world coordinate
def img_cor_2_world_cor():
    x_img, y_img, z_img = img_cor_points[0]
    xc, yc, zc = x_img*z_img, y_img*z_img, z_img
    p_cam = np.array([xc, yc, zc])
    xw, yw, zw = np.dot(np.linalg.inv(k), p_cam)
    print(xw, yw, zw)
    print(x, y, z)

In [6]:
# Now begin the initialization of dictionary for image training set

In [107]:
X_MAX = 3383
Y_MAX = 2709
class stage2Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, mode = 'train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train = pd.read_csv(csv_file)
        if mode == 'train':
            self.train = self.train[:int(self.train.shape[0]*0.7)]
        else:
            self.train = self.train[int(self.train.shape[0]*0.7):]
    def __len__(self):
        return self.train.shape[0]

    def __getitem__(self, idx):
        x_l = 1.02 # augment this
        y_l = 0.80 # augment this
        z_l = 2.31 # augment this
        # also do data augmentation later
        # try directly training on quaternion?
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = train.loc[idx]['ImageId']
        pred_string = train.loc[idx]['PredictionString']
        img = plt.imread('./train_images/' + img_name + '.jpg').astype(float)/255
        mg_y, mg_x = np.meshgrid(np.linspace(0, 1, img.shape[0]), np.linspace(0, 1, img.shape[1]), indexing = 'ij')
        #process prediction string
        items = pred_string.split(' ')
        model_types, yaws, pitches, rolls, xs, ys, zs = [items[i::7] for i in range(7)]
        sample = []
        #iterate over each vehcicle
        for yaw, pitch, roll, x, y, z in zip(yaws, pitches, rolls, xs, ys, zs):
            yaw, pitch, roll, x, y, z = [float(x) for x in [yaw, pitch, roll, x, y, z]]
            img_x, img_y, img_z = np.dot(k, [x, y, z])
            #target = np.array([yaw, np.sin(pitch), np.cos(pitch), rotate(roll, np.pi), img_x/10000, img_y/10000, img_z/10]) #result to regress to
            target = np.array([yaw, np.sin(pitch), np.cos(pitch), rotate(roll, np.pi), x/5, y/5, z/5]) #result to regress to
            # I think the pitch and yaw should be exchanged
            yaw, pitch, roll = -pitch, -yaw, -roll
            Rt = np.eye(4)
            t = np.array([x, y, z])
            Rt[:3, 3] = t
            Rt[:3, :3] = euler_to_Rot(yaw, pitch, roll).T
            Rt = Rt[:3, :]
            P = np.array([[0, 0, 0, 1],
                          [x_l, y_l, -z_l, 1],
                          [x_l, y_l, z_l, 1],
                          [-x_l, y_l, z_l, 1],
                          [-x_l, y_l, -z_l, 1],
                          [x_l, -y_l, -z_l, 1],
                          [x_l, -y_l, z_l, 1],
                          [-x_l, -y_l, z_l, 1],
                          [-x_l, -y_l, -z_l, 1]]).T

            # call this function before change the astype
            img_cor_points = np.dot(k, np.dot(Rt, P)) # calculate 8 corners and center in terms of camera coordinate
            img_cor_points = img_cor_points.T
            img_cor_points[:, 0] /= img_cor_points[:, 2] # project onto 2d camera plane
            img_cor_points[:, 1] /= img_cor_points[:, 2]
            x_max = int(np.amax(img_cor_points[:,0])) # calculate 2d box
            if x_max>X_MAX:
                x_max = X_MAX
            x_min = int(np.amin(img_cor_points[:,0]))
            if x_min<0:
                x_min = 0
            y_max = int(np.amax(img_cor_points[:,1]))
            if y_max>Y_MAX:
                y_max = Y_MAX
            y_min = int(np.amin(img_cor_points[:,1])) 
            if y_min<0:
                y_min = 0
            # Crop image
            img_crop = cv2.resize(img[y_min:y_max, x_min:x_max], (224, 224))
            mg_x_crop = cv2.resize(mg_x[y_min:y_max, x_min:x_max], (224, 224))[..., np.newaxis]
            mg_y_crop = cv2.resize(mg_y[y_min:y_max, x_min:x_max], (224, 224))[..., np.newaxis]
            input_tensor = np.concatenate((img_crop, mg_y_crop, mg_x_crop), axis = 2).transpose((2, 0, 1))
            sample.append((input_tensor, target))
        return sample

In [108]:
stage2set = stage2Dataset('./train.csv')
trainloader = torch.utils.data.DataLoader(stage2set, batch_size=1, shuffle = True)

In [109]:
from torchvision.models import resnet18
net = resnet18()
net.add_module('conv1', torch.nn.Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False))
net.add_module('fc', torch.nn.Linear(in_features=512, out_features=7, bias=True))
#net = ResidualAttentionModel() #this is the attention version, which is much slower
net.train()
optimizer = torch.optim.Adam(net.parameters())
criterion = torch.nn.MSELoss()
for parameter in net.parameters():
    if len(parameter.shape)>1:
        torch.nn.init.xavier_uniform_(parameter, gain=1.0)

In [ ]:
for batch_idx, sample in enumerate(trainloader):
    X = torch.zeros([len(sample), 5, 224, 224])
    Y = torch.zeros([len(sample), 7])
    for sample_idx, (input_tensor, target) in enumerate(sample):
        X[sample_idx] = input_tensor[0]
        Y[sample_idx] = target[0]
    optimizer.zero_grad()
    begin = time.time()
    outputs = net(X)
    print(outputs.shape)
    angleloss = criterion(outputs[:, :4], Y[:, :4])
    xyzloss = criterion(outputs[:, 4:], Y[:, 4:])
    loss = angleloss*10+xyzloss
    loss.backward()
    optimizer.step()
    #print(time.time()-begin)
    print(angleloss.detach().numpy(), xyzloss.detach().numpy())
    #pred_xyz = torch.Tensor(np.dot(np.linalg.inv(k), (outputs[:, 4:].detach().numpy()*[10000, 10000, 10]).T).T/5)
    #true_xyz = torch.Tensor(np.dot(np.linalg.inv(k), (Y[:, 4:].detach().numpy()*[10000, 10000, 10]).T).T/5)
    #print('metric loss is ', criterion(pred_xyz, true_xyz).detach().numpy())

torch.Size([6, 7])
8.003484 86.17624
torch.Size([9, 7])
8.219187 75.163315
torch.Size([19, 7])
2.213333 54.99658
torch.Size([14, 7])
4.2958927 10.843459
torch.Size([16, 7])
4.959213 11.103041
torch.Size([6, 7])
2.6610754 32.60711
torch.Size([14, 7])
4.9812603 21.617899
torch.Size([15, 7])
2.647082 24.230507
torch.Size([21, 7])
1.760177 25.09544
torch.Size([15, 7])
1.0581515 13.647569
torch.Size([24, 7])
0.68319833 3.7828379
torch.Size([13, 7])
0.84319645 194.7646
torch.Size([2, 7])
0.20357168 18.952545
torch.Size([15, 7])
1.2134074 11.51234
torch.Size([14, 7])
1.3070426 8.362022
torch.Size([6, 7])
0.75093955 5.9327693
torch.Size([7, 7])
0.34694892 9.645549
torch.Size([10, 7])
0.3124387 35.497105
torch.Size([19, 7])
0.57914436 25.01891
torch.Size([7, 7])
0.7708785 13.599396
torch.Size([5, 7])
0.3241258 5.140794
torch.Size([10, 7])
0.340809 12.299566
torch.Size([9, 7])
0.23680952 18.124155
torch.Size([8, 7])
0.40784702 15.88563
torch.Size([7, 7])
0.49992952 9.200259
torch.Size([8, 7])
0.

torch.Size([17, 7])
0.398613 5.053494
torch.Size([7, 7])
0.20061794 5.267253
torch.Size([5, 7])
0.40331075 7.9488792
torch.Size([5, 7])
0.34282535 6.65731
torch.Size([10, 7])
0.22363552 3.8312955
torch.Size([17, 7])
0.25974703 6.6578484
torch.Size([12, 7])
0.15448992 6.5439987
torch.Size([9, 7])
0.3383858 13.831065
torch.Size([13, 7])
0.15284939 3.242401
torch.Size([13, 7])
0.23848476 2.0955808
torch.Size([4, 7])
0.37603632 3.4813323
torch.Size([13, 7])
0.14710952 1.933305
torch.Size([12, 7])
0.11232763 4.3892326
torch.Size([9, 7])
0.18776648 20.742273
torch.Size([12, 7])
0.15482473 0.9963785
torch.Size([20, 7])
0.25071347 6.4239473
torch.Size([15, 7])
0.24557415 4.3856616
torch.Size([10, 7])
0.13490623 1.4455221
torch.Size([5, 7])
0.30587083 13.951525
torch.Size([9, 7])
0.1840702 12.421504
torch.Size([6, 7])
0.2719435 3.2020988
torch.Size([12, 7])
0.15880352 6.3787484
torch.Size([9, 7])
0.15417555 6.2168427
torch.Size([7, 7])
0.22105473 12.123264
torch.Size([11, 7])
0.1787383 1.096644

In [ ]:
sum([p.numel() for p in net.parameters()])

In [93]:
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co